In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import sys

import pandas as pd
import numpy as np

import time

from collections import deque

import torch

import copy

from tqdm.auto import tqdm

In [2]:
from datasets import load_original_dataset, load_deleted_dataset
from models import CNN

In [3]:
DATA_DIR = 'Datasets/Features/'
EPOCHS = 5
PERCENTAGES = [1, 10, 20, 30, 40, 50, 60, 70, 80, 90, 99]

In [4]:
# For MNIST dataset: python3 main.py --bz 16384 --epochs 20 --model Logistic_regression --dataset MNIST --wd 0.0001 --lr 0.1 0.05 --lrlen 10 10 --method deltagrad --period 5 --init 20 -m 2 --cached_size 20
is_GPU = True
device = 0
BATCH_SIZE = 1024
LR = 0.05
WD = 0.0001
INIT_EPOCHS = 1
PERIOD = 2
M = 2

In [5]:
sys.path.append(os.path.abspath('./libraries/DeltaGrad/src/'))

In [6]:
from utils import post_processing_gradien_para_list_all_epochs, append_gradient_list, init_model, get_model_para_shape_list, get_devectorized_parameters, get_all_vectorized_parameters1, compute_model_para_diff, compute_derivative_one_more_step
from main_delete import explicit_iters, compute_grad_final3, cal_approx_hessian_vec_prod0_3, compute_approx_hessian_vector_prod_with_prepared_terms1, prepare_hessian_vec_prod0_3

In [7]:
# https://github.com/thuwuyinjun/DeltaGrad/blob/master/src/main_delete.py#L362
def update_para_final2(para, gradient_list, alpha):
    
    vec_para = get_all_vectorized_parameters1(para)
    
    vec_para -= alpha*gradient_list
        
    return vec_para

# https://github.com/thuwuyinjun/DeltaGrad/blob/master/src/Models/DNN_single.py#L105
class DGCNN(CNN):
    def get_all_gradient(self):
        
        para_list = []
        
        for param in self.parameters():
            para_list.append(param.grad.clone())
            
        return para_list    

In [8]:
# https://github.com/thuwuyinjun/DeltaGrad/blob/master/src/main_delete.py#L1176

In [9]:
# https://github.com/thuwuyinjun/DeltaGrad/blob/master/README.md?plain=1#L77
# update the model after the training phase with deltagrad

# https://github.com/thuwuyinjun/DeltaGrad/blob/master/src/main.py#L107

def fit(model, save_dir, train_set, test_set, forget_set):
    
    os.makedirs(save_dir, exist_ok=True)
    
    # https://github.com/thuwuyinjun/DeltaGrad/blob/master/src/Models/Data_preparer.py#L325
    # replace softmax+nlloss with cross_entropy
    criterion = torch.nn.CrossEntropyLoss()
    
    # prepare model
    # https://github.com/thuwuyinjun/DeltaGrad/blob/master/src/main_delete.py#L1133
    net = copy.deepcopy(model)
    
    # https://github.com/thuwuyinjun/DeltaGrad/blob/master/src/Models/Data_preparer.py#L326
    optimizer = torch.optim.SGD(model.parameters(), lr=LR, weight_decay=WD)
    net_optimizer = torch.optim.SGD(net.parameters(), lr=LR, weight_decay=WD)

    train_batch_size = int(np.ceil(BATCH_SIZE * len(train_set) / (len(train_set) + len(forget_set))))
    forget_batch_size = int(np.ceil(BATCH_SIZE * len(forget_set) / (len(train_set) + len(forget_set))))

    num_steps = min(len(train_set) // train_batch_size, len(forget_set) // forget_batch_size)

    train_x, train_y = train_set.tensors[0], train_set.tensors[1]
    forget_x, forget_y = forget_set.tensors[0], forget_set.tensors[1]

    # https://github.com/thuwuyinjun/DeltaGrad/blob/master/src/main_delete.py#L1202
    # https://github.com/thuwuyinjun/DeltaGrad/blob/master/src/main_delete.py#L815

    # https://github.com/thuwuyinjun/DeltaGrad/blob/master/src/main_delete.py#L819
    para = list(model.parameters())
    # https://github.com/thuwuyinjun/DeltaGrad/blob/master/src/main_delete.py#L826
    full_shape_list, shape_list, total_shape_size = get_model_para_shape_list(model.parameters())

    # https://github.com/thuwuyinjun/DeltaGrad/blob/master/src/main_delete.py#L836
    S_k_list = deque()
    Y_k_list = deque()
    
    train_times = list()
    
    train_accs, test_accs, forget_accs = list(), list(), list()
    
    for epoch in range(EPOCHS):    
        
        # train
        
        train_time = 0
        
        start_time = time.time()

        model.train()
        for i in range(num_steps):

            # https://github.com/thuwuyinjun/DeltaGrad/blob/master/src/main_delete.py#L890

            batch_remaining_X = train_x[train_batch_size*i:train_batch_size*(i+1)].cuda()
            batch_remaining_Y = train_y[train_batch_size*i:train_batch_size*(i+1)].cuda()

            # https://github.com/thuwuyinjun/DeltaGrad/blob/master/src/main_delete.py#L930

            batch_delta_X = forget_x[forget_batch_size*i:forget_batch_size*(i+1)].cuda()
            batch_delta_Y = forget_y[forget_batch_size*i:forget_batch_size*(i+1)].cuda()

            # https://github.com/thuwuyinjun/DeltaGrad/blob/master/src/main_delete.py#L895

            curr_matched_ids_size = batch_delta_X.shape[0]

            # https://github.com/thuwuyinjun/DeltaGrad/blob/master/src/model_train.py#L18
            
            net_optimizer.zero_grad()
            output = net(torch.concat([batch_remaining_X, batch_delta_X], dim=0))
            loss = criterion(output, torch.concat([batch_remaining_Y, batch_delta_Y], dim=0))
            loss.backward()
            net_optimizer.step()

            gradient_list = []
            para_list = []
            append_gradient_list(gradient_list, None, para_list, net, None, is_GPU, device)

            para_list_tensor, grad_list_tensor = post_processing_gradien_para_list_all_epochs(para_list, gradient_list)
            para_list_tensor, grad_list_tensor = para_list_tensor.cuda(), grad_list_tensor.cuda()
                
            # https://github.com/thuwuyinjun/DeltaGrad/blob/master/src/main_delete.py#L927

            if epoch < INIT_EPOCHS:

                para, _, init_hessian_para_prod, theta_k = explicit_iters(
                    batch_delta_X, batch_delta_Y, batch_remaining_X, batch_remaining_Y, 
                    curr_matched_ids_size, model, para, epoch, i, M+1, S_k_list, Y_k_list, LR, WD, 
                    grad_list_tensor, grad_list_tensor, 0, full_shape_list, shape_list, 
                    is_GPU, device, 
                    criterion, optimizer, None, None
                )

            else:
                
                '''use l-bfgs algorithm to evaluate the gradients'''

                # https://github.com/thuwuyinjun/DeltaGrad/blob/master/src/main_delete.py#L952
                
                init_model(model, para)

                compute_derivative_one_more_step(model, batch_delta_X, batch_delta_Y, criterion, optimizer)
                
                gradient_dual = model.get_all_gradient()
                
                with torch.no_grad():
                
                    vec_para_diff = torch.t((get_all_vectorized_parameters1(para) - para_list_tensor))
                    
                    if (epoch - INIT_EPOCHS) / PERIOD >= 1:
                        if (epoch - INIT_EPOCHS) % PERIOD == 0:
                            zero_mat_dim, curr_Y_k, curr_S_k, sigma_k, mat_prime = prepare_hessian_vec_prod0_3(list(S_k_list)[1:], list(Y_k_list)[1:], i, INIT_EPOCHS, M, is_GPU, device)
                            
                            mat = np.linalg.inv(mat_prime.cpu().numpy())
                            mat = torch.from_numpy(mat)
                            mat = mat.to(device)
                            
                        hessian_para_prod = compute_approx_hessian_vector_prod_with_prepared_terms1(zero_mat_dim, curr_Y_k, curr_S_k, sigma_k, mat, vec_para_diff, is_GPU, device)
                        
                    else:
                        '''S_k_list, Y_k_list, v_vec, k, is_GPU, device'''
                        hessian_para_prod, zero_mat_dim, curr_Y_k, curr_S_k, sigma_k, mat_prime = cal_approx_hessian_vec_prod0_3(list(S_k_list)[1:], list(Y_k_list)[1:], vec_para_diff, M, is_GPU, device)
                    
                    is_positive, final_gradient_list = compute_grad_final3(
                        get_all_vectorized_parameters1(para), torch.t(hessian_para_prod), 
                        get_all_vectorized_parameters1(gradient_dual), 
                        grad_list_tensor, para_list_tensor, 
                        batch_remaining_X.shape[0] + curr_matched_ids_size, curr_matched_ids_size, 
                        LR, WD, is_GPU, device
                    )
                        
                    vec_para = update_para_final2(para, final_gradient_list, LR)
                    
                    para = get_devectorized_parameters(vec_para, full_shape_list, shape_list)
                

        
        train_time += time.time() - start_time
            
        train_times.append(train_time)
        
        # test
            
        model.eval()
        with torch.no_grad():
            
            #
            
            x, y = train_set.tensors
            
            accs = list()
            
            for i in range(0, x.shape[0], BATCH_SIZE):
            
                output = model(x[i:i+BATCH_SIZE].cuda())

                predicted = torch.argmax(output.data, dim=-1)
                accs.append((predicted == y[i:i+BATCH_SIZE].cuda()).float().mean().detach().cpu().numpy())
            
            train_accs.append(np.mean(accs))
            
            #
            
            x, y = test_set.tensors
            
            accs = list()
            
            for i in range(0, x.shape[0], BATCH_SIZE):
            
                output = model(x[i:i+BATCH_SIZE].cuda())

                predicted = torch.argmax(output.data, dim=-1)
                accs.append((predicted == y[i:i+BATCH_SIZE].cuda()).float().mean().detach().cpu().numpy())
            
            test_accs.append(np.mean(accs))
            
            #

            x, y = forget_set.tensors
            
            accs = list()

            for i in range(0, x.shape[0], BATCH_SIZE):

                output = model(x[i:i+BATCH_SIZE].cuda())

                predicted = torch.argmax(output.data, dim=-1)
                accs.append((predicted == y[i:i+BATCH_SIZE].cuda()).float().mean().detach().cpu().numpy())

            forget_accs.append(np.mean(accs))
        
        # save
        torch.save(model.state_dict(), os.path.join(save_dir, f'{(epoch+1):03d}.pt'))

    return train_times, train_accs, test_accs, forget_accs

In [10]:
results = list()

for percentage in tqdm(PERCENTAGES):

    # https://github.com/thuwuyinjun/DeltaGrad/blob/master/src/main_delete.py#L1124
    # https://github.com/thuwuyinjun/DeltaGrad/blob/master/src/Models/DNN_single.py#L47
    # remove softmax
    model = DGCNN().cuda()
        
    # https://github.com/thuwuyinjun/DeltaGrad/blob/master/src/main_delete.py#L1133
    model.load_state_dict(torch.load('./weights/init.pt'))
    
    train_set, test_set, forget_set = load_deleted_dataset(DATA_DIR, percentage)
    
    train_times, train_accs, test_accs, forget_accs = fit(model, f'weights/DeltaGrad/{percentage}', train_set, test_set, forget_set)
    
    df = pd.DataFrame(zip(train_times, train_accs, test_accs, forget_accs), columns=['train_time', 'train_acc', 'test_acc', 'forget_acc'])
    df['epoch'] = range(1, EPOCHS+1)
    df['percentage'] = percentage
    
    results.append(df)

results = pd.concat(results).set_index(['percentage', 'epoch'])

results.to_csv(f'results/DeltaGrad.csv')

  0%|          | 0/11 [00:00<?, ?it/s]

In [11]:
results

train_time  train_acc  test_acc  forget_acc
percentage epoch                                             
1          1        2.798030   0.684852  0.692215    0.691667
           2        2.019802   0.097077  0.098184    0.096667
           3        1.956517   0.097077  0.098184    0.096667
           4        1.979096   0.097077  0.098184    0.096667
           5        1.321297   0.097077  0.098184    0.096667
10         1        2.231315   0.222672  0.220566    0.224216
           2        2.166035   0.098714  0.098184    0.098991
           3        2.136526   0.098714  0.098184    0.098991
           4        2.113361   0.098714  0.098184    0.098991
           5        1.438504   0.098714  0.098184    0.098991
20         1        2.225544   0.585216  0.586904    0.583623
           2        2.249819   0.098268  0.098184    0.100724
           3        2.200084   0.098268  0.098184    0.100724
           4        2.176492   0.098268  0.098184    0.100724
           5        1.501146   0.098268  0.098184    0.100724
30         1        2.222276   0.794317  0.809740    0.786604
           2        2.346921   0.097726  0.098184    0.098793
           3        2.300222   0.097726  0.098184    0.098793
           4        2.300544   0.097726  0.098184    0.098793
           5        1.557895   0.097726  0.098184    0.098793
40         1        2.249781   0.105284  0.106583    0.105236
           2        2.413353   0.098573  0.098184    0.099080
           3        2.431201   0.098573  0.098184    0.099080
           4        2.385542   0.098573  0.098184    0.099080
           5        1.656636   0.098573  0.098184    0.099080
50         1        2.193686   0.098578  0.098184    0.098468
           2        2.475509   0.098578  0.098184    0.098468
           3        2.451005   0.098578  0.098184    0.098468
           4        2.387493   0.098578  0.098184    0.098468
           5        1.726167   0.098578  0.098184    0.098468
60         1        2.216934   0.354998  0.351469    0.354704
           2        2.569359   0.099406  0.098184    0.098210
           3        2.568106   0.099406  0.098184    0.098210
           4        2.573989   0.099406  0.098184    0.098210
           5        1.815980   0.099406  0.098184    0.098210
70         1        2.203233   0.310677  0.307605    0.306548
           2        2.710657   0.098867  0.098184    0.100725
           3        2.771116   0.098867  0.098184    0.100725
           4        2.674032   0.098867  0.098184    0.100725
           5        2.052062   0.098867  0.098184    0.100725
80         1        2.236583   0.099429  0.098184    0.098532
           2        2.763274   0.099429  0.098184    0.098532
           3        2.816030   0.099429  0.098184    0.098532
           4        2.756171   0.099429  0.098184    0.098532
           5        2.109777   0.099429  0.098184    0.098532
90         1        2.259893   0.097831  0.098184    0.098866
           2        2.903175   0.097831  0.098184    0.098866
           3        2.863327   0.097831  0.098184    0.098866
           4        2.812195   0.097831  0.098184    0.098866
           5        2.139593   0.097831  0.098184    0.098866
99         1        2.111590   0.096667  0.098184    0.097077
           2        2.681713   0.096667  0.098184    0.097077
           3        2.716931   0.096667  0.098184    0.097077
           4        2.699249   0.096667  0.098184    0.097077
           5        2.062720   0.096667  0.098184    0.097077